In [4]:
import pandas as pd
import numpy as np
import os
# import seaborn as sns
# import plotly.express as px
# import plotly.graph_objs as go
# from plotly.tools import FigureFactory as FF
import re
import os
from datetime import datetime, timedelta

df = pd.DataFrame()
os.getcwd()

# df = pd.read_clipboard(header = None)
# df[0] = pd.to_datetime(df[0].astype(str), yearfirst=True)
# px.line(df, x= 0, y = 1)

'c:\\Users\\dscshap3808\\Documents\\my_scripts_new\\play4_activity'

In [11]:
# df = pd.read_csv('./data/siemens_1012.csv')
df = pd.read_csv('./data/coach_1108.csv', sep  = '\001')
re1 = re.compile(r'(?<=\.).+')
df.columns = [re1.findall(i)[0] for i in list(df.columns.to_numpy())]
df = df.dropna(subset=['activity_start_time'])

In [12]:
df['activity_end_time'] = pd.to_datetime(df['activity_end_time'])
df['activity_start_time'] = pd.to_datetime(df['activity_start_time'].fillna(0))
df['create_time'] = pd.to_datetime(df['create_time'])

In [13]:
"""
time_gap_inner d单个activity的耗费时间.
"""
df['time_gap_inner'] = df['activity_end_time']-df['activity_start_time']
df['time_gap_inner'] = df['time_gap_inner'].fillna(
    timedelta(0)
    ).apply(timedelta.total_seconds)
df['time_gap_inner'] = [0 if i < 0 else i for i in df['time_gap_inner']]

"""
time_gap_outer 前一个activity到后一个activity相差的时间\. (单个时间内会有大量activity发生.)
act type is new mission;
"""
df = df.sort_values(['activity_end_time', 'user_id'])
df['time_gap_outer'] = df\
    .groupby(['user_id','inc_day'], dropna = False)['activity_end_time'].transform('diff')
df['time_gap_outer'] = df['time_gap_outer'].fillna(timedelta(0)).apply(timedelta.total_seconds)
df['time_gap_outer'] = [0 if i< 0 else i for i in df['time_gap_outer']]

"""
time gap of the day. 总计活跃时间, ( 未减除动作间的时间间隔 )
"""
mid1 = df.groupby(['inc_day', 'user_id']).agg(
    a = ('activity_start_time','min'), 
    b  = ('activity_start_time', 'max'))\
        [['a','b']].diff(axis = 1)['b'].apply(timedelta.total_seconds).reset_index()
mid1.columns = ['inc_day','user_id','time_gap_today']
df = df.merge(mid1, on = ['inc_day', 'user_id'], how = 'left')


In [14]:
def get_hour():
    list = ['activity_start_time', 'activity_end_time','create_time']
    
    for i in list: 
        listnew = i + '_hour'
        df[listnew] = df[i].dt.round('min').astype(str).str.slice(11,)
    return df
df = get_hour() 

In [15]:
df['activity_start_time_sec'] = df['activity_start_time'].astype(str).str.slice(11,)
df['activity_end_time_sec'] = df['activity_end_time'].astype(str).str.slice(11,)

In [17]:
# pd.set_option("display.max_rows", None, "display.max_columns", 55)
# df[df['user_id'] == 'BJMARS003'].query('time_gap_outer != 0')[[
#     'activity_start_time', 'activity_end_time','time_gap_inner'	,'time_gap_outer','time_gap_today']].\
#         sort_values('activity_start_time', ascending=False)

In [44]:
df[df['user_id'] == 'ADC002RF'].time_gap_today.unique()

array([25323.])

In [46]:
cals = df.groupby(['user_id', 'inc_day', 'ou_code', 'activity_type']).agg(
    time_inner_sum = ('time_gap_inner', 'sum'),
    time_outer_sum = ('time_gap_outer', 'sum'),
    time_today_sum = ('time_gap_today', 'mean'),
    tt_qty = ('qty', 'sum')).reset_index()
cals['total_eff'] = cals.groupby(['user_id', 'inc_day', 'ou_code'])['time_inner_sum']\
    .transform('sum') / cals['time_today_sum']

In [73]:
cals['inner_eff']    = cals['time_inner_sum'] / cals['tt_qty']
cals['rested_rate']  = cals['time_outer_sum'] / cals['time_today_sum']
cals['actived_rate'] = cals['time_inner_sum'] / cals['time_today_sum']
cals = cals.fillna(0)

In [74]:
cals.to_csv('./data/cals_coach.csv', index = None)

In [47]:
# df[['ou_code',   'wms_warehouse_id', 'user_id', 'activity_type' ,
#        'activity_start_time', 'activity_end_time',   
#        'activity_id',  'lpn','sku_code', 'qty',
#        'time_gap_inner','time_gap_outer','time_gap_today',
#        'activity_start_time_hour', 'activity_end_time_hour', 'create_time_hour']]\
#               .query("user_id == 'DGMHE' & activity_start_time == '2021-09-26 00:00:35' ")

In [48]:
# pd.set_option("display.max_rows", 15, "display.max_columns", 15)
# df[df['user_id'] == 'DGMHE'].query("time_gap_outer > 10.0") 

def check(id):
    # print("===", df[df['user_id'] == id]['ou_name'].iloc[0],"===" , '\n', 'usr_id = ',id)
    for i, j  in enumerate([1,10, 30, 60, 600, 14400]):
        q = "time_gap_outer > " + str(j) 
        print(
            "和上个动作时间差距为" , j, '到' , [1,10, 30, 60, 600, 14400, '14400+'][i+1] , 
            '秒的动作数为: ', df[df['user_id'] == id].query(q).shape[0]) 

In [49]:
check('DSCSZVP0162')

和上个动作时间差距为 1 到 10 秒的动作数为:  0
和上个动作时间差距为 10 到 30 秒的动作数为:  0
和上个动作时间差距为 30 到 60 秒的动作数为:  0
和上个动作时间差距为 60 到 600 秒的动作数为:  0
和上个动作时间差距为 600 到 14400 秒的动作数为:  0
和上个动作时间差距为 14400 到 14400+ 秒的动作数为:  0


In [18]:
df[['ou_code', 'wms_warehouse_id', 'user_id', 'activity_type', 'activity_sub_type',
       'activity_start_time', 'activity_end_time', 'lpn', 'sku_code',
       'order_id', 'from_location', 'to_location', 'qty', 'create_time',
       'activity_id', 'inc_day',
       'time_gap_inner','time_gap_outer','time_gap_today',
       'activity_start_time_hour', 'activity_end_time_hour',
       'create_time_hour','activity_start_time_sec', 'activity_end_time_sec']].\
              to_csv('./activi_for.csv', index=None, encoding='utf_8_sig')

In [ ]:
# df.groupby(['inc_day', 'user_id']).agg(
#     a = ('activity_start_time','min'), 
#     b  = ('activity_start_time', 'max'))[['a','b']].diff(axis = 1)['b']\
# .apply(timedelta.total_seconds).reset_index()
# df[df['user_id'] == 'DSCSZXP0325'][[
# 'activity_start_time','time_gap_inner','time_gap_outer','time_gap_today'
# ]].sort_values('activity_start_time')

In [31]:
data = df[['ou_code', 'wms_warehouse_id', 'user_id', 'activity_type', 'activity_sub_type',
       'activity_start_time', 'activity_end_time', 'lpn', 'sku_code',
       'order_id', 'from_location', 'to_location', 'qty', 'create_time',
       'activity_id', 'inc_day',
       'time_gap_inner','time_gap_outer','time_gap_today',
       'activity_start_time_hour', 'activity_end_time_hour',
       'create_time_hour','activity_start_time_sec', 'activity_end_time_sec']]

data[[ 'sku_code']] = data[[ 'sku_code']].fillna(0).astype(int)

In [ ]:
import statsmodels.api

In [42]:
data['time_gap_inner'].value_counts()

0.0       24735
1.0        5440
4.0        3225
3.0        3016
5.0        2863
          ...  
3057.0        1
2223.0        1
1623.0        1
2511.0        1
376.0         1
Name: time_gap_inner, Length: 784, dtype: int64

In [1]:
# px.histogram(data['time_gap_inner'])

In [43]:
data['time_gap_outer'].value_counts()

0.0       17208
5.0        5568
6.0        4181
1.0        3786
4.0        3230
          ...  
758.0         1
936.0         1
8285.0        1
975.0         1
2102.0        1
Name: time_gap_outer, Length: 874, dtype: int64

In [2]:
# px.histogram(data['time_gap_outer'])